In [ ]:
!pip install sdv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66

In [ ]:
import sdv
print(sdv.__version__)

1.19.0


In [ ]:
import pandas as pd
import numpy as np
from sdv.lite import SingleTablePreset
from sklearn.preprocessing import StandardScaler
import joblib
import os
from datetime import datetime
from sdv.lite import SingleTablePreset
from sdv.metadata import SingleTableMetadata

from sdv.single_table import TVAESynthesizer
from sdv.metadata import SingleTableMetadata

In [ ]:
def encode_categorical_features(df, target_column):
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    categorical_cols = [col for col in categorical_cols if col != target_column]
    df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=False)
    return df_encoded, categorical_cols



In [ ]:
def decode_categorical_features(df_synth, original_df, target_column):
    decoded_df = df_synth.copy()
    cat_columns = original_df.select_dtypes(include=['object', 'category']).columns.tolist()
    cat_columns = [col for col in cat_columns if col != target_column]

    for col in cat_columns:
        one_hot_cols = [c for c in df_synth.columns if c.startswith(col + '_')]
        if not one_hot_cols:
            continue

        decoded_df[col] = df_synth[one_hot_cols].idxmax(axis=1).str[len(col)+1:]
        decoded_df.drop(columns=one_hot_cols, inplace=True)

    return decoded_df



In [ ]:


def train_tvae_model(df, epochs=100):
    """
    Train a synthetic data model using TVAE with specified number of epochs.

    Args:
        df (pd.DataFrame): The real dataset.
        epochs (int): Number of training iterations.

    Returns:
        model: Trained TVAE model
    """
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data=df)

    model = TVAESynthesizer(metadata=metadata, epochs=epochs)
    model.fit(df)
    return model



In [ ]:
def generate_synthetic_data(tvae, scaler, original_df, feature_names, target_column, num_rows):
    synthetic_data = tvae.sample(num_rows)
    numeric_cols = [col for col in synthetic_data.columns if col in feature_names and synthetic_data[col].dtype != 'object' and col != target_column]
    synthetic_data[numeric_cols] = scaler.inverse_transform(synthetic_data[numeric_cols])

    # Decode one-hot encoded features
    decoded_df = decode_categorical_features(synthetic_data, original_df, target_column)
    decoded_df[target_column] = synthetic_data[target_column].values
    return decoded_df



In [ ]:
def save_files(model, synthetic_df, model_dir='models', data_dir='synthetic_data'):
    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(data_dir, exist_ok=True)

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    model_path = os.path.join(model_dir, f'tvae_model_{timestamp}.pkl')
    data_path = os.path.join(data_dir, f'synthetic_data_{timestamp}.csv')

    joblib.dump(model, model_path)
    synthetic_df.to_csv(data_path, index=False)

    print(f"✅ Model saved to: {model_path}")
    print(f"✅ Synthetic data saved to: {data_path}")

    return model_path, data_path

In [ ]:
# Load data
df = pd.read_csv("customer_churn_dataset-training-master.csv")
target_column = "Churn"

tvae_model = train_tvae_model(df)
synthetic_data = tvae_model.sample(num_rows=200000)
# Save model and synthetic data
model_path, data_path = save_files(tvae_model, synthetic_data)


/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


✅ Model saved to: models/tvae_model_20250413_153946.pkl
✅ Synthetic data saved to: synthetic_data/synthetic_data_20250413_153946.csv


In [ ]:
synthetic_data = tvae_model.sample(num_rows=500000)
# Save model and synthetic data
model_path, data_path = save_files(tvae_model, synthetic_data)

✅ Model saved to: models/tvae_model_20250413_155017.pkl
✅ Synthetic data saved to: synthetic_data/synthetic_data_20250413_155017.csv
